In [1]:
from qreduce.cs_vqe import cs_vqe_model
from qreduce.utils import * 
from itertools import combinations
import json
import numpy as np

In [2]:
with open('data/model_data.json', 'r') as infile:
        mol_data = json.load(infile)
mol=mol_data['B+_STO-3G_SINGLET']
ham=mol['ham']
n_q=mol['num_qubits']
set_nc = mol['terms_noncon']
cs_vqe_mol = cs_vqe_model(ham, n_q, noncontextual_set = set_nc)

{'ZZZIZ': 0, 'ZIZIZ': 2, 'ZZIIZ': 4, 'ZZZII': 1, 'ZZZZZ': 3}
{0: {'generator': 'ZZZIZ', 'symmetry': True, 'rotation': [('YIIII', 1.5707963267948966, True), ('YZZIZ', 1.5707963267948966, True)]}, 1: {'generator': 'ZZZII', 'symmetry': True, 'rotation': [('IYIII', 1.5707963267948966, True), ('ZYZII', 1.5707963267948966, True)]}, 2: {'generator': 'ZIZIZ', 'symmetry': True, 'rotation': [('IIYII', 1.5707963267948966, True), ('ZIYIZ', 1.5707963267948966, True)]}, 3: {'generator': 'ZZZZZ', 'symmetry': False, 'rotation': [('IIIYI', 2.3215009309522018e-08, False), ('IIIYI', 1.5707963267948966, True), ('ZZZYZ', 1.5707963267948966, True)]}, 4: {'generator': 'ZZIIZ', 'symmetry': True, 'rotation': [('IIIIY', 1.5707963267948966, True), ('ZZIIY', 1.5707963267948966, True)]}}
[('IIIYI', 2.3215009309522018e-08, False), ('IIIYI', 1.5707963267948966, True), ('ZZZYZ', 1.5707963267948966, True), ('YIIII', 1.5707963267948966, True), ('YZZIZ', 1.5707963267948966, True), ('IYIII', 1.5707963267948966, True), ('

In [3]:
print("Generators:", cs_vqe_mol.generators)
print("Clique reps:", cs_vqe_mol.cliquereps)
match=mol['X_index']==cs_vqe_mol.anti_clique_qubit
print('Anti-clique qubit match?', match, '\n')

print("G assignment:", cs_vqe_mol.nu)#, ' | original:', np.array(list(mol['G'].values()), dtype=int))
print("r vector:", cs_vqe_mol.r)
print("Noncon nrg:", cs_vqe_mol.ngs_energy)
print('NGS matches original?', abs(cs_vqe_mol.ngs_energy-mol['noncon'])<1e-13, '\n')

print("Stabilizers:", cs_vqe_mol.stabilizers)
print("Stab qubits:", cs_vqe_mol.stab_qubits)

Generators: ['ZZZIZ', 'ZIZIZ', 'ZZIIZ', 'ZZZII']
Clique reps: ['ZZZXZ', 'ZZZZZ']
Anti-clique qubit match? True 

G assignment: [-1 -1 -1  1]
r vector: [2.32150093e-08 1.00000000e+00]
Noncon nrg: -24.1489885988536
NGS matches original? True 

Stabilizers: {'ZIIII': 1, 'IIZIZ': -1, 'ZZZIZ': 1, 'IZIIZ': 1, 'IIIZI': -1.0}
Stab qubits: [0, 2, 0, 1, 3]


In [4]:
cs_vqe_mol.ngs_energy,mol['noncon']

(-24.1489885988536, -24.14898859885359)

In [5]:
num_sim_q = mol['chem_acc_num_q']
exact = exact_gs_energy(cs_vqe_mol.ham)
print('Exact energy:',exact)
print('Noncon error:', cs_vqe_mol.ngs_energy-exact)
print(f'Target_error for {num_sim_q} qubits:', exact_gs_energy(mol['ham_reduced'][num_sim_q])-exact)

Exact energy: -24.189264917100314
Noncon error: 0.04027631824671474
Target_error for 3 qubits: 0.0001845717122392898


In [6]:
for comb in combinations(range(n_q), num_sim_q+1):
    comb = list(comb)# mol['order'][0:mol['chem_acc_num_q']] #=list(comb)
    ham_cs = cs_vqe_mol.contextual_subspace_hamiltonian_2(comb)
    #print(ham_cs)
    cs_energy = exact_gs_energy(ham_cs)
    in_range = cs_energy<cs_vqe_mol.ngs_energy and cs_energy>exact
    print(f'error for qubits {comb}:', cs_energy-exact, '| In expected range?', in_range)
    #print(ham_cs)

LinAlgError: 0-dimensional array given. Array must be at least two-dimensional

In [ ]:
cs_vqe_mol.stabilizer_rotations

In [ ]:
'IZIZZ'.find('X')

In [ ]:
from itertools import permutations

In [70]:
ops_to_rotate = ['ZZZIZ', 'ZIZIZ', 'ZZIIZ', 'ZZZII']
ops_to_rotate = ['ZZZZZ']+sorted(ops_to_rotate, key=lambda x:x.count('Z'))

In [71]:
num_qubits=5
used_qubits = []
clique_rot = rotate_operator(cs_vqe_mol.anti_clique_operator, [
                                           cs_vqe_mol.unitary_partitioning])
stabilizer_map = {i: {'generator': None, 'symmetry': True,
                         'rotation': []} for i in range(num_qubits)}
rotations=[]
for G in ops_to_rotate:
    print(rotations)
    print(G)
    print(rotate_operator({G:1}, rotations))
    G_rot = list(rotate_operator({G:1}, rotations).keys())[0]

    if G_rot.count('Z')==1:
        used_qubits.append(G_rot.index('Z'))
        if G_rot in clique_rot:
            rotations[Z_index]['symmetry'] = False
            rotations[Z_index]['rotation'].append(self.unitary_partitioning)
    else:
        if set(G_rot) in [{'Z'},{'I','Z'}]:
            Z_indices = [i for i in range(cs_vqe_mol.num_qubits) if G_rot[i]=='Z' and i not in used_qubits]
            Z_index = Z_indices[0]
            rot_op = amend_string_index(['I' for i in range(cs_vqe_mol.num_qubits)],Z_index,'Y')
            stabilizer_map[Z_index]['rotation'].append((rot_op, np.pi/2, True))
            rotations.append((rot_op, np.pi/2, True))
            G_offdiag = amend_string_index(G_rot,Z_index,'X')
        else:
            G_offdiag = deepcopy(G_rot)
            Z_index = [i for i in range(cs_vqe_mol.num_qubits) if G_offdiag[i] in [
                        'X', 'Y'] and i not in used_qubits][0]
        
        stabilizer_map[Z_index]['generator'] = G
        if G_offdiag[Z_index] == 'X':
            rot_op = amend_string_index(G_offdiag,Z_index,'Y')
        else:
            rot_op = amend_string_index(G_offdiag,Z_index,'X')
        stabilizer_map[Z_index]['rotation'].append((rot_op, np.pi/2, True))
        rotations.append((rot_op, np.pi/2, True))
        
        if G in clique_rot:
            stabilizer_map[Z_index]['rotation'].insert(0, cs_vqe_mol.unitary_partitioning)
            stabilizer_map[Z_index]['symmetry'] = False
            rotations.insert(0,cs_vqe_mol.unitary_partitioning)
            
    used_qubits.append(Z_index)
    print('')

[]
ZZZZZ
{'ZZZZZ': 1}

[('IIIYI', 2.3215009309522018e-08, False), ('YIIII', 1.5707963267948966, True), ('YZZZZ', 1.5707963267948966, True)]
ZIZIZ
{'ZZIZI': -1}

[('IIIYI', 2.3215009309522018e-08, False), ('YIIII', 1.5707963267948966, True), ('YZZZZ', 1.5707963267948966, True), ('IYIII', 1.5707963267948966, True), ('ZYIZI', 1.5707963267948966, True)]
ZZIIZ
{'ZIZZI': -1}

[('IIIYI', 2.3215009309522018e-08, False), ('YIIII', 1.5707963267948966, True), ('YZZZZ', 1.5707963267948966, True), ('IYIII', 1.5707963267948966, True), ('ZYIZI', 1.5707963267948966, True), ('IIYII', 1.5707963267948966, True), ('ZIYZI', 1.5707963267948966, True)]
ZZZII
{'ZIIZZ': -1}

[('IIIYI', 2.3215009309522018e-08, False), ('YIIII', 1.5707963267948966, True), ('YZZZZ', 1.5707963267948966, True), ('IYIII', 1.5707963267948966, True), ('ZYIZI', 1.5707963267948966, True), ('IIYII', 1.5707963267948966, True), ('ZIYZI', 1.5707963267948966, True), ('IIIYI', 1.5707963267948966, True), ('ZIIYZ', 1.5707963267948966, True)]
ZZ

In [76]:
rotate_operator({'ZIZIZ':1}, stabilizer_map[1]['rotation'])

{'ZIZIZ': 1}

In [73]:
rotations

[('IIIYI', 2.3215009309522018e-08, False),
 ('YIIII', 1.5707963267948966, True),
 ('YZZZZ', 1.5707963267948966, True),
 ('IYIII', 1.5707963267948966, True),
 ('ZYIZI', 1.5707963267948966, True),
 ('IIYII', 1.5707963267948966, True),
 ('ZIYZI', 1.5707963267948966, True),
 ('IIIYI', 1.5707963267948966, True),
 ('ZIIYZ', 1.5707963267948966, True),
 ('IIIIY', 1.5707963267948966, True),
 ('IIIZY', 1.5707963267948966, True)]

In [60]:
order=[3,0,1,2,4]
rotations=[]
for i in order:
    rotations+=stabilizer_map[i]['rotation']

In [62]:
rotations

[('IIIYI', 2.3215009309522018e-08, False),
 ('IIIYI', 1.5707963267948966, True),
 ('IIIYZ', 1.5707963267948966, True),
 ('YIIII', 1.5707963267948966, True),
 ('YIZIZ', 1.5707963267948966, True),
 ('IYIII', 1.5707963267948966, True),
 ('ZYZII', 1.5707963267948966, True),
 ('IIYII', 1.5707963267948966, True),
 ('IZYIZ', 1.5707963267948966, True),
 ('IIIIY', 1.5707963267948966, True),
 ('IIZIY', 1.5707963267948966, True)]